# Modeling

## Load Packages

In [1]:
from operator import mod
from os import getcwd
from os.path import exists, join

import joblib
from sklearn.datasets import fetch_california_housing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVR
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport

In [2]:
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression, LinearRegression
import warnings
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import  GradientBoostingClassifier
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC 
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import recall_score

from sklearn import tree
from sklearn.decomposition import PCA, SparsePCA

from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
import json
import pickle
from IPython.display import Image
import warnings

In [3]:
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from collections import Counter

In [4]:
import warnings
warnings.filterwarnings('ignore')

## Load Data

In [5]:
cdc_survey = pd.read_csv('../data/cdc_nhanes_survey_responses_clean.csv')

# filter to pregnant moms
cdc_survey_pmom = cdc_survey[cdc_survey['has_been_pregnant'] == 1]
cdc_survey_pmom

,Unnamed: 0,SEQN,ALQ111,ALQ121,ALQ130,ALQ142,ALQ270,ALQ280,ALQ290,ALQ151,...,has_health_insurance,has_health_insurance_gap,has_smoked_tabacco_last_5days,is_male,is_usa_born,has_diabetes,has_overweight_diagnosis,has_tried_to_lose_weight_12mo,has_been_pregnant,monthly_poverty_index
5,5,109284.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,0,0.0,0.0,1.0,1.0,1.0,NaN
7,7,109290.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,...,1.0,0.0,0.0,0,1.0,1.0,0.0,1.0,1.0,4.80
8,8,109291.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0,1.0,0.0,1.0,0.0,1.0,NaN
11,11,109295.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,1.0,4.37
14,14,109300.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,...,1.0,0.0,0.0,0,0.0,0.0,0.0,1.0,1.0,2.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8950,8950,124802.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,...,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,1.0,NaN
8951,8951,124803.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0,0.0,1.0,0.0,0.0,1.0,1.18
8957,8957,124812.0,1.0,6.000000e+00,3.0,8.000000e+00,10.0,9.0,5.397605e-79,2.0,...,0.0,NaN,0.0,0,1.0,0.0,0.0,0.0,1.0,3.66
8958,8958,124813.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,...,1.0,1.0,0.0,0,0.0,0.0,0.0,1.0,1.0,NaN


## Training Step

### Features to Use

In [48]:
all_columns = [
    # Target
    'MDD',
    # Depression screener
    'little_interest_in_doing_things',
    'feeling_down_depressed_hopeless',
    'trouble_falling_or_staying_asleep',
    'feeling_tired_or_having_little_energy',
    'poor_appetitie_or_overeating',
    'feeling_bad_about_yourself',
    'trouble_concentrating',
    'moving_or_speaking_to_slowly_or_fast',
    'thoughts_you_would_be_better_off_dead',
    'difficult_doing_daytoday_tasks',
    # Alcohol & smoking
    'has_smoked_tabacco_last_5days',
    'alcoholic_drinks_past_12mo',    
    # Diet & Nutrition
    'how_healthy_is_your_diet',    
    'count_lost_10plus_pounds',
    'has_tried_to_lose_weight_12mo',       
    # Physical health & Medical History
    'count_days_seen_doctor_12mo',
    'duration_last_healthcare_visit',        
    'count_days_moderate_recreational_activity',   
    'count_minutes_moderate_recreational_activity',
    'count_minutes_moderate_sedentary_activity',
    'general_health_condition',    
    'has_diabetes',
    'has_overweight_diagnosis',         
    # Demographic data
    'food_security_level_household',   
    'food_security_level_adult',    
    'monthly_poverty_index_category',
    'monthly_poverty_index',
    'count_hours_worked_last_week',
    'age_in_years',   
    'education_level',
    'is_usa_born',    
    'has_health_insurance',
    'has_health_insurance_gap'   
]
len(all_columns)

# cols removed since all null
# count_tried_to_lose_weight_youth
# count_days_physical_activity_youth

34

In [50]:
# find columns where everything is null
# need to delete those
#cdc_survey_pmom = cdc_survey_pmom[all_columns]
#columns = cdc_survey_pmom.columns[cdc_survey_pmom.notnull().any() == False]
#columns

### Create Test & Train Datasets

In [51]:
def get_model_data(original_df, columns, test_size_prop=0.2):
    """
    Function to build feature & indicator matrices for both train & test.
    """
    
    df_to_use = original_df[columns]
    
    # Create test & train data
    x = df_to_use.iloc[:,1:].values
    y = df_to_use['MDD'].values
    
    # SimpleImputer() = fill in missing values
    # note imputer may drop columns if no values exist for it
    imputer = SimpleImputer(strategy='median')  
    x = imputer.fit_transform(x)

    # RobustScaler() = scale features to remove outliers
    trans = RobustScaler()
    x = trans.fit_transform(x)

    x_train, x_test, y_train, y_test = train_test_split(
        x, 
        y, 
        test_size=test_size_prop, 
        random_state=42
    ) 
    
    return x_train, x_test, y_train, y_test

## Test Step

In [23]:
def plot_confusion_matrix(y_test, pred_labels):
    """
    Function that displays a confusion matrix for provided true and predicted classes
    """
    #print(f'cover type 1 and type 2 total correct {np.sum(np.diag(metrics.confusion_matrix(y_test, pred_labels))[:2])}')

    cm = confusion_matrix(y_test, pred_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    fig, ax = plt.subplots(figsize=(5,5))
    disp = disp.plot(include_values=True, cmap='viridis', ax=ax, xticks_rotation='horizontal')    
    plt.grid(False)
    plt.show()
    return

def get_performance_df(label_actual, label_pred, index_label):
    """
    Function to calculate performance metrics for model.
    Includes precision, recal, F1, & support.
    """
    performance_report = classification_report(label_actual, label_pred, output_dict=True)
    performance_report = performance_report['macro avg']
    result_table = pd.DataFrame(performance_report, index = [index_label])
    return result_table

def baseline_models(
    x_train, 
    y_train, 
    x_test, 
    y_test,
    do_smote=True,
    show_confusion_matrix=False,
    show_score_dataframe=False):
    """
    Function that trains and makes predictions using 5 of the classifiers went over during the class.
    Meant as a helper function for easier testing of different modeling pipelines.
    """

    #  do_smote
    if do_smote == True:
        sm = SMOTE(random_state=42)
        x_train, y_train = sm.fit_resample(x_train, y_train)

    # K-Nearest Neighbors
    knn = KNeighborsClassifier()
    knn.fit(x_train, y_train)
    pred_labels_knn  = knn.predict(x_test)
    score_knn = get_performance_df(y_test, pred_labels_knn,'Knn')
    
    # Logistic Regression
    lm = LogisticRegression()
    lm.fit(x_train, y_train)
    pred_labels_lr  = lm.predict(x_test)
    score_lr = get_performance_df(y_test, pred_labels_lr,'Logistic Regression')
        
    # Bernoulii Naive Bayes
    bnb = BernoulliNB()
    bnb.fit(x_train, y_train)
    pred_labels_bnb  = bnb.predict(x_test)
    score_bnb = get_performance_df(y_test, pred_labels_bnb,'Bernoulli Naive Bayes')    
        
    # Gaussian Naive Bayes
    gnb = GaussianNB()
    gnb.fit(x_train, y_train)
    pred_labels_gnb  = gnb.predict(x_test)
    score_gnb = get_performance_df(y_test, pred_labels_gnb,'Gaussian Naive Bayes')    

    # Random Forest
    rf = RandomForestClassifier(random_state=0)
    rf.fit(x_train, y_train)
    pred_labels_rf  = rf.predict(x_test)
    predictions_posterior_rf = rf.predict_proba(x_test)
    score_rf = get_performance_df(y_test, pred_labels_rf,'Random Forest')      

    # make dataframe with scores
    scores = pd.concat([score_knn, score_lr, score_bnb, score_gnb, score_rf])
    scores = scores.sort_values(by = 'recall', ascending=False)
    
    if show_score_dataframe:
        display(scores.style.set_table_attributes('style="font-size: 17px"').hide_index())
    
    if show_confusion_matrix:
        print('\nK-Nearest Neighbors Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_knn)
        print('Logistic Regression Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_lr)
        print('Bernoulli Naive Bayes Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_bnb)
        print('Gaussian Naive Bayes Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_gnb)
        print('Random Forest Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_rf)

    return scores

### Baseline models (just depression screener)

In [52]:
x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = all_columns[0:11] # just depression screener
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(2729, 10)
(683, 10)
(2729,)
(683,)


In [53]:
baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model[['precision','recall','f1-score']]

,precision,recall,f1-score
Bernoulli Naive Bayes,0.609595,0.672248,0.627428
Gaussian Naive Bayes,0.619179,0.657007,0.633433
Logistic Regression,0.653195,0.524981,0.525183
Random Forest,0.552096,0.512648,0.506990
Knn,0.516733,0.506154,0.500418


In [54]:
baseline_model_with_smote = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model_with_smote[['precision','recall','f1-score']]

,precision,recall,f1-score
Logistic Regression,0.590391,0.687960,0.598980
Bernoulli Naive Bayes,0.587726,0.677093,0.597050
Gaussian Naive Bayes,0.600932,0.675809,0.617413
Knn,0.587408,0.638338,0.600637
Random Forest,0.510287,0.505500,0.502845


### Baseline models (all variables)

In [55]:
x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = all_columns
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(2729, 33)
(683, 33)
(2729,)
(683,)


In [56]:
baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model[['precision','recall','f1-score']]

,precision,recall,f1-score
Bernoulli Naive Bayes,0.620125,0.688796,0.640166
Gaussian Naive Bayes,0.613735,0.664627,0.630484
Logistic Regression,0.588745,0.515895,0.510511
Knn,0.552096,0.512648,0.506990
Random Forest,0.450808,0.498377,0.473400


In [57]:
baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model[['precision','recall','f1-score']]

,precision,recall,f1-score
Logistic Regression,0.594492,0.700451,0.603120
Gaussian Naive Bayes,0.586315,0.687306,0.590851
Bernoulli Naive Bayes,0.598842,0.674186,0.614727
Knn,0.566361,0.657116,0.555784
Random Forest,0.641773,0.536659,0.544620


## Hyperprameter grid search
SVG is terrible, but need to do some gridsearch on above baselines

In [11]:

# Model Pipeline
processing_pipeline = make_pipeline(SimpleImputer(), RobustScaler(), SVR())

params = {
    "simpleimputer__strategy": ["mean", "median"],
    "robustscaler__quantile_range": [(25.0, 75.0), (30.0, 70.0)],
    "svr__C": [0.1, 1.0],
    "svr__gamma": ["auto", 0.1],
}

grid = GridSearchCV(processing_pipeline, param_grid=params, n_jobs=-1, cv=5, verbose=3)

model_filename = "model_pipeline.pkl"
model_path = join(getcwd(), model_filename)
print(model_path)
grid.fit(X_train, y_train)

print(f"Train R^2 Score : {grid.best_estimator_.score(X_train, y_train):.3f}")
print(f"Test R^2 Score : {grid.best_estimator_.score(X_test, y_test):.3f}")
print(f"Best R^2 Score Through Grid Search : {grid.best_score_:.3f}")
print(f"Best Parameters : {grid.best_params_}")


/Users/orencarmeli/Desktop/capstone210-ppd-carmeli_chari_hand_lee/modeling/model_pipeline.pkl
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Train R^2 Score : 0.514
Test R^2 Score : -0.013
Best R^2 Score Through Grid Search : 0.039
Best Parameters : {'robustscaler__quantile_range': (25.0, 75.0), 'simpleimputer__strategy': 'median', 'svr__C': 1.0, 'svr__gamma': 'auto'}
